### Loading _not quantized_ ML models

In [ ]:
from tensorflow.keras.models import load_model
###### AE MODEL LOAD ######
ae_wo_classifier = load_model('model/AE_model/KERAS_check_best_model.h5')
# ae_wo_classifier_pruned = load_model('model/AE_model/KERAS_check_pruned_best_model.h5')
ae_wo_classifier_pruned = load_model('model/AE_model/KERAS_check_pruned_best_model.model')
ae_w_classifier = load_model('model/AE_model/KERAS_check_best_model_classifier.h5')
ae_w_classifier_pruned = load_model('model/AE_model/KERAS_check_pruned_best_model_classifier.h5')

### Loading _quantized_ ML models

In [ ]:
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects
co = {}
_add_supported_quantized_objects(co)
###### QAE MODEL LOAD ######

#qae_wo_classifier = load_model('model/QAE_model/KERAS_check_best_model.h5', custom_objects=co)
qae_wo_classifier_pruned = load_model('model/QAE_model/KERAS_check_pruned_best_model.h5', custom_objects=co)
#qae_w_classifier = load_model('model/QAE_model/KERAS_check_best_model_classifier.h5', custom_objects=co)

#qae_w_classifier_pruned = load_model('model/QAE_model/KERAS_check_pruned_best_model_only_classifier.h5', custom_objects=co)
qae_w_classifier_pruned = load_model('model/QAE_model/KERAS_check_pruned_best_model_classifier1.h5', custom_objects=co)

## Check sparsity
Make a quick check that the model was indeed trained sparse. We'll just make a histogram of the weights of the 1st layer, and hopefully observe a large peak in the bin containing '0'. Note logarithmic y axis.

In [ ]:
sum = 0
sum_pru = 0

for i in [2,3,5]:
    for j in [0,1]:
        w= ae_w_classifier.layers[i].weights[j].numpy()
        sum +=w.size 
        w_prun = ae_w_classifier_pruned.layers[i].weights[j].numpy()
        sum_pru += w.size - np.sum(w_prun==0)

        if j==0:
            print('weights:')
        else:
            print('bias:')
        
        print(w.size,'\n', w.size - np.sum(w_prun==0))

    print("layer"+str(i),'\n')
print (sum, '\n',  sum -sum_pru)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

w = 10
h = 10
fig = plt.figure(figsize=(10, 10))
columns = 2
rows = 4
fig.subplots_adjust(hspace=0.4, wspace=0.4)

num_layers = [1,2,3,4,7,9,11]#,11,14,15]
#this applies for the quantized loop
num_layers = [1,3,5,7,8,10,12]#,11,14,15]


j = 1

for i in num_layers:
#for i in range(1, columns*rows +1):  
    fig.add_subplot(rows, columns, j)
    w = qae_w_classifier_pruned.layers[i].weights[0].numpy()
    h, b = np.histogram(w, bins=100)
    plt.bar(b[:-1], h, width=b[1]-b[0])
    plt.semilogy()
    plt.title('Weight distribution of layer '+ str(j))
    j += 1

plt.savefig('./Report/images/section4/ae_pruned_weightsdistr.png')
plt.show()


# Make an hls4ml config & model

In [ ]:
import plotting
import hls4ml
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

config = hls4ml.utils.config_from_keras_model(qae_w_classifier_pruned, granularity='name')
config['LayerName']['encoder_input']['Precision']='ap_ufixed<6,0>'

#config['LayerName']['classifier_output']['Precision'] = 'ap_fixed<32,12>'
config['LayerName']['classifier_output']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['classifier_output']['inv_table_t'] = 'ap_fixed<18,4>'
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(qae_w_classifier_pruned,
                                                       hls_config=config,
                                                       output_dir='model/QAE_model/hls4ml_prj',
                                                       part='xcu250-figd2104-2L-e')
                                                       #part='xcvu9p-flgc2104aaz')
                                                       #part='xc7vx690t-ffg176-2')


# Compile

In [ ]:
hls_model.compile()

# Build

In [ ]:
hls_model.build(csim=True, cosim=True)

### Prediction results

In [ ]:
import MNIST_database as mnist

#Choose the final size of your image dataset
size_final = 8

data_zoom = mnist.MNISTData(size_initial=20, size_final=size_final, color_depth=5, flat=True)

from matplotlib import pyplot as plt

#           0  1  2  3  4  5  6  7  8  9          
num_list = [3, 2, 1, 32,4, 15,21,0, 61,12] 



In [ ]:
#img = np.flip(data_zoom.x_test[num_list[3]].reshape(1,-1))
#img = data_zoom.x_test[32].reshape(1,-1)
#plt.imshow(img.reshape(size_final,size_final), cmap='gray_r')

In [ ]:
#img = []
ax = plt.subplot(1, 1 , 1)
img = data_zoom.x_test[num_list[0]].reshape(1,-1)

plt.imshow(img.reshape(size_final,size_final), cmap='gray_r')
print(img)
plt.imshow(img.reshape(size_final,size_final), cmap='gray_r')

In [ ]:
import time
import numpy as np
img = data_zoom.x_test[num_list[1]].reshape(1,-1)
plt.imshow(img.reshape(size_final,size_final), cmap='gray_r')
start_time = time.time()
out = qae_w_classifier_pruned.predict(img)
print("--- %s seconds ---" % (time.time() - start_time))
print(out)
print(np.sum(out))

In [ ]:
y = hls_model.predict(img)[1]
